# API Extraction and SQL Loader for Springer Lab - LT3

In [1]:
import numpy as np
import pandas as pd
import sqlite3
import requests
import os
import json
from datetime import datetime
import json
import ast
import re
import ipywidgets as widgets
import functools
from IPython.display import clear_output

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from scipy import sparse


Code is to cleanup the database, counter and log file in case we want to do fresh loads.

In [1]:
# if os.path.isfile('springer.db'):
#     os.remove('springer.db')
# if os.path.isfile('ctr.txt'):
#     os.remove('ctr.txt')
# if os.path.isfile('log.txt'):
#     os.remove('log.txt')

Create SQL table we can define in the `table_name` the table name

In [59]:
table_name = 'springer_dump_data_science'
def create_springer_dump_table(conn):
    sql = """
          CREATE TABLE IF NOT EXISTS """ + table_name + """(
              identifier TEXT,
              contentType TEXT,
              language TEXT,
              url TEXT,
              title TEXT,
              creators TEXT,
              bookEditors TEXT,
              publicationName TEXT,
              openaccess TEXT,
              doi TEXT,
              publisher TEXT,
              publicationDate TEXT,
              publicationType TEXT,
              printIsbn TEXT,
              electronicIsbn TEXT,
              isbn TEXT,
              genre TEXT,
              onlineDate TEXT,
              copyright TEXT,
              abstract TEXT,
              conferenceInfo TEXT,
              keyword TEXT,
              coverDate TEXT,
              issn TEXT,
              eIssn TEXT,
              volume TEXT,
              number TEXT,
              issueType TEXT,
              topicalCollection TEXT,
              startingPage TEXT,
              endingPage TEXT,
              journalId TEXT,
              response TEXT,
              subjects TEXT
          )
          """
    conn.execute(sql)
    conn.commit()
    

Function `load_springer_data` will run the data loads from the springer API. <br>
<br>
`api_key` should be defined to run the job<br>
`ctr.txt` content should be an integer which index will be fetched from the Springer API<br>
`doc_c` variable that controls the number of rows to be loaded<br>
`search` parameter should be a string value and you can use the q extensions of the springer API based on the documentation<br>
<br>
This automatically restarts from the last ctr.txt value as this gets updated every loop you can also double check the log file.

In [60]:
def load_springer_data(search):
    
    if os.path.isfile('ctr.txt') is False:
        with open('ctr.txt', 'w') as f:
            f.write('1')
        
    f = open('ctr.txt', "r")
    s = int(f.read())
    doc_c = 25_001
    p = 50
    dic = {
            'q': search,
            'p': p,
            's': s,
            'api_key': '' #
          }
    
    lst = []
    print(f'Load Started {datetime.now()}')
    
    conn = sqlite3.connect('springer.db')
    create_springer_dump_table(conn)
    while True:
        print(f'Current index {s} Time: {datetime.now()}', end='\r')
        js = (requests
                 .get(f'https://api.springernature.com/metadata/json',
                      params=dic
                 ).json())
        for i in js['records']:
        
            for h in i:
                if type(i.get(h)) is list:
                    i[h] = (str(i.get(h))) 
            df_load = pd.DataFrame(i, index=[0])
            df_load.to_sql(table_name, con=conn, if_exists='append', 
                           index=False)
        with open('ctr.txt', 'w') as f:
            f.write(str(s))
        with open('log.txt', 'a') as f:
            f.write(json.dumps(dic) + '\n')
        s += p
        dic['s'] = s
        if s >= doc_c:
            break
    print(f'Load Ended {datetime.now()}')
    conn.close()
    return 

Call the `load_springer_data` and pass the parameter here we used `keyword:Data Science` to get all the relevant documents with this keyword.

In [61]:
js = load_springer_data('keyword:Data Science')

Load Started 2022-11-20 15:30:40.016073
Load Ended 2022-11-20 16:00:12.33364916:00:08.464295


Data Cleanup for the database

In [56]:
# conn = sqlite3.connect('springer.db')
# conn.execute('DROP TABLE springer_dump_data_science')
# conn.commit()

Check the data loads

In [36]:
conn = sqlite3.connect('springer.db')
df = pd.read_sql('select * from springer_dump_data_science', conn)
print(len(df))
display(df.head(5))
df = pd.read_sql('select DISTINCT * from springer_dump_data_science LIMIT 25000', conn)
print(len(df))
display(df.head(5))
print(df.columns)

25000


,identifier,contentType,language,url,title,creators,bookEditors,publicationName,openaccess,doi,...,eIssn,volume,number,issueType,topicalCollection,startingPage,endingPage,journalId,response,subjects
0,doi:10.1007/s41095-022-0275-7,Article,en,"[{'format': '', 'platform': '', 'value': 'http...",A survey of urban visual analytics: Advances a...,"[{'creator': 'Deng, Zikun'}, {'creator': 'Weng...",None,Computational Visual Media,true,10.1007/s41095-022-0275-7,...,None,9,1,None,None,3,39,41095,None,"['Computer Science', 'Computer Graphics', 'Use..."
1,doi:10.1007/978-1-0716-2659-7_30,Chapter Protocol,en,"[{'format': '', 'platform': '', 'value': 'http...",Best Practices to Promote Data Utility and Reu...,"[{'creator': 'Dunlap, Garrett S.'}, {'creator'...",None,Salamanders,false,10.1007/978-1-0716-2659-7_30,...,None,None,None,None,None,None,None,None,None,"['Life Sciences', 'Biological Techniques']"
2,doi:10.1007/978-981-19-4960-9_67,Chapter ConferencePaper,en,"[{'format': '', 'platform': '', 'value': 'http...",Study of Relevance of the Engineering Career i...,"[{'creator': 'Molina-Granja, Fernando'}, {'cre...",None,Inventive Communication and Computational Tech...,false,10.1007/978-981-19-4960-9_67,...,None,None,None,None,None,None,None,None,None,"['Engineering', 'Computational Intelligence', ..."
3,doi:10.1007/978-3-031-20322-0_12,Chapter ConferencePaper,en,"[{'format': '', 'platform': '', 'value': 'http...",Data Mining Prospective Associated with the Pu...,"[{'creator': 'Cruz, José Quintana'}, {'ORCID':...",None,New Perspectives in Software Engineering,false,10.1007/978-3-031-20322-0_12,...,None,None,None,None,None,None,None,None,None,"['Engineering', 'Computational Intelligence', ..."
4,doi:10.1007/978-3-031-20601-6_53,Chapter ConferencePaper,en,"[{'format': '', 'platform': '', 'value': 'http...",Fog of Things Framework to Handle Data Streami...,"[{'creator': 'Zahran, Sherif'}, {'creator': 'E...",None,Proceedings of the 8th International Conferenc...,false,10.1007/978-3-031-20601-6_53,...,None,None,None,None,None,None,None,None,None,"['Engineering', 'Computational Intelligence', ..."


25000


,identifier,contentType,language,url,title,creators,bookEditors,publicationName,openaccess,doi,...,eIssn,volume,number,issueType,topicalCollection,startingPage,endingPage,journalId,response,subjects
0,doi:10.1007/s41095-022-0275-7,Article,en,"[{'format': '', 'platform': '', 'value': 'http...",A survey of urban visual analytics: Advances a...,"[{'creator': 'Deng, Zikun'}, {'creator': 'Weng...",None,Computational Visual Media,true,10.1007/s41095-022-0275-7,...,None,9,1,None,None,3,39,41095,None,"['Computer Science', 'Computer Graphics', 'Use..."
1,doi:10.1007/978-1-0716-2659-7_30,Chapter Protocol,en,"[{'format': '', 'platform': '', 'value': 'http...",Best Practices to Promote Data Utility and Reu...,"[{'creator': 'Dunlap, Garrett S.'}, {'creator'...",None,Salamanders,false,10.1007/978-1-0716-2659-7_30,...,None,None,None,None,None,None,None,None,None,"['Life Sciences', 'Biological Techniques']"
2,doi:10.1007/978-981-19-4960-9_67,Chapter ConferencePaper,en,"[{'format': '', 'platform': '', 'value': 'http...",Study of Relevance of the Engineering Career i...,"[{'creator': 'Molina-Granja, Fernando'}, {'cre...",None,Inventive Communication and Computational Tech...,false,10.1007/978-981-19-4960-9_67,...,None,None,None,None,None,None,None,None,None,"['Engineering', 'Computational Intelligence', ..."
3,doi:10.1007/978-3-031-20322-0_12,Chapter ConferencePaper,en,"[{'format': '', 'platform': '', 'value': 'http...",Data Mining Prospective Associated with the Pu...,"[{'creator': 'Cruz, José Quintana'}, {'ORCID':...",None,New Perspectives in Software Engineering,false,10.1007/978-3-031-20322-0_12,...,None,None,None,None,None,None,None,None,None,"['Engineering', 'Computational Intelligence', ..."
4,doi:10.1007/978-3-031-20601-6_53,Chapter ConferencePaper,en,"[{'format': '', 'platform': '', 'value': 'http...",Fog of Things Framework to Handle Data Streami...,"[{'creator': 'Zahran, Sherif'}, {'creator': 'E...",None,Proceedings of the 8th International Conferenc...,false,10.1007/978-3-031-20601-6_53,...,None,None,None,None,None,None,None,None,None,"['Engineering', 'Computational Intelligence', ..."


Index(['identifier', 'contentType', 'language', 'url', 'title', 'creators',
       'bookEditors', 'publicationName', 'openaccess', 'doi', 'publisher',
       'publicationDate', 'publicationType', 'printIsbn', 'electronicIsbn',
       'isbn', 'genre', 'onlineDate', 'copyright', 'abstract',
       'conferenceInfo', 'keyword', 'coverDate', 'issn', 'eIssn', 'volume',
       'number', 'issueType', 'topicalCollection', 'startingPage',
       'endingPage', 'journalId', 'response', 'subjects'],
      dtype='object')


Integration of springer database since we have done multiple parallel loads.

In [103]:
conn = sqlite3.connect('springer_data_science.db')
conn2 = sqlite3.connect('springer.db')
conn3 = sqlite3.connect('springer_2.db')
conn4 = sqlite3.connect('springer_3.db')
conn5 = sqlite3.connect('springer_4.db')

# df = pd.read_sql('select DISTINCT * from springer_dump_data_science LIMIT 25000', conn2)
# df.to_sql('springer_dump', con=conn, if_exists='append', index=False)

# df = pd.read_sql('select DISTINCT * from springer_dump_data_science LIMIT 25000', conn3)
# df.to_sql('springer_dump', con=conn, if_exists='append', index=False)

# df = pd.read_sql('select DISTINCT * from springer_dump_data_science LIMIT 25000', conn4)
# df.to_sql('springer_dump', con=conn, if_exists='append', index=False)

# df = pd.read_sql('select DISTINCT * from springer_dump_data_science LIMIT 25000', conn5)
# df.to_sql('springer_dump', con=conn, if_exists='append', index=False)

df = pd.read_sql('select DISTINCT * from springer_dump', conn)
print(len(df))


100000
